In [25]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

# GSDC Raw Data Overview

In [ ]:
glob.glob("./data/raw_data/*")

In [ ]:
all_compounds_screened = pd.read_csv("data/raw_data/all_compounds_screened.csv")
all_cellines_screened = pd.read_excel("data/raw_data/all_cellines_screened.xlsx", sheet_name=0)
all_experiment = pd.read_excel("data/raw_data/GDSC2_drug_dose_cellines_IC50s.xlsx")

In [ ]:
# 选取有测序结果和甲基化数据等数据的细胞系
filtered_celline = all_cellines_screened.loc[
    (all_cellines_screened['Whole Exome Sequencing (WES)'] == "Y") &
    (all_cellines_screened['Methylation'] == "Y") &
    (all_cellines_screened['Gene Expression'] == "Y") &
    (all_cellines_screened['Copy Number Alterations (CNA)'] == "Y") &
    (all_cellines_screened['Drug\nResponse'] == "Y")
]
filtered_celline.head()

In [ ]:
filtered_celline.head()

In [ ]:
a = set(all_experiment['CELL_LINE_NAME'])
b = set(filtered_celline['Sample Name'])
print(f"experiment sheet includes {a.__len__()} unqiue cellines")
print(f"all_celline sheet includes {b.__len__()} unique cellines")
celline_barcode = list(a.intersection(b))
print(f"two sheets have {celline_barcode.__len__()} overlapping cellines")

# GDSC Single Celline Multi-omics data Overview

## SNV data (Mutation Data)

In [ ]:
snv_df = pd.read_csv('data/raw_data/mutations_all_20230202.csv', low_memory=False)
model_list = pd.read_csv('data/raw_data/model_list_20230306.csv', low_memory=False)
snv_df.drop(columns='model_name', inplace=True)

In [ ]:
snv_df = snv_df.join(model_list[['model_id', 'model_name']].set_index('model_id'), on="model_id", how="inner")

In [ ]:
snv_df.reset_index(drop=True, inplace=True)

 important_only = ['cds_disrupted','complex_sub','downstream', 'ess_splice','frameshift','missense','nonsense','silent','splice_region','start_lost','stop_lost','upstream']


'cds_disrupted', 终止密码子突变
 'complex_sub', 复合物替换突变
 'downstream', 下游
 'ess_splice', 外显子剪辑沉默
 'frameshift', 移码突变
 'inframe', 开放框架内突变
 'intronic', 内显子
 'missense', 错义突变
 'nc_ess_splice', 
 'nc_variant', 
 'nonsense', 无义突变
 'silent', 沉默突变
 'splice_region', 剪辑区域
 'start_lost', 启动子丢失
 'stop_lost', 终止密码子丢失
 'upstream' 上游

In [ ]:
important_only = ['cds_disrupted','complex_sub','downstream', 'ess_splice','frameshift','missense','nonsense','silent','splice_region','start_lost','stop_lost','upstream']

In [ ]:
snv_df = snv_df[snv_df['effect'].isin(important_only)]

In [ ]:
df_table = pd.pivot_table(data=snv_df, 
                          index='model_name', 
                          columns='gene_symbol', 
                          values='effect',
                          aggfunc='count',
                          fill_value=0)

## Methylation Data

In [ ]:
met_df = pd.read_csv('data/raw_data/F2_METH_CELL_DATA.txt', sep = '\t', index_col=0)
met_df.head()

In [ ]:
met_df = met_df.T

In [ ]:
met_lookup = pd.read_excel('data/raw_data/methSampleId_2_cosmicIds.xlsx', sheet_name=0)

In [ ]:
met_lookup['Sentrix_Barcode'] = list("_".join([i,j]) for i, j in zip(met_lookup['Sentrix_ID'].astype(str), met_lookup['Sentrix_Position']))

In [ ]:
tb = dict(zip(met_lookup['Sentrix_Barcode'], met_lookup['Sample_Name']))
met_df.rename(columns=tb, inplace=True)
met_df = met_df.T

## Copy Number Variation and FPKM file readin 

In [ ]:
all_cna = pd.read_csv("./data/raw_data/celline_SNP6_cnv_gistics_20191101/cnv_gistic_20191101.csv", low_memory=False,
                      skiprows=lambda x: x in [0, 2])
all_fpkm = pd.read_csv("./data/raw_data/cellines_rnaseq_all_20220624/rnaseq_fpkm_20220624.csv", low_memory=False,
                       skiprows=lambda x: x in [0, 2, 3, 4])

## Datasets Alignment

In [ ]:
all_fpkm = all_fpkm.drop(columns=['model_name'])
all_fpkm.rename(columns={"Unnamed: 1": "celline_barcode"}, inplace=True)
all_fpkm.set_index('celline_barcode', inplace=True)
all_fpkm = all_fpkm.T
all_fpkm.fillna(0.0, inplace=True)

In [ ]:
all_cna = all_cna.drop(columns=['model_name'])
all_cna.rename(columns={"Unnamed: 1": "celline_barcode"}, inplace=True)
all_cna.set_index('celline_barcode', inplace=True)
all_cna = all_cna.T
all_cna.fillna(0.0, inplace=True)

In [ ]:
s1 = set(all_fpkm.index).intersection(celline_barcode)
s2 = set(all_cna.index).intersection(celline_barcode)
celline_barcode = list(s1.intersection(s2))
print(f"Two datasets have {len(celline_barcode)} common cellines")

In [ ]:
common_genes = list(set(cna_df.columns).intersection(fpkm_df.columns))
print(f"Two datasets have {len(common_genes)} genes(features)")

In [ ]:
all_fpkm.loc[celline_barcode].to_csv("./data/processed_data/fpkm.csv", sep='\t')
all_cna.loc[celline_barcode].to_csv("./data/processed_data/cna.csv", sep='\t')
all_fpkm = all_fpkm.loc[celline_barcode]
all_cna = all_cna.loc[celline_barcode]

## GDSC Dataset

Filter dataset

In [ ]:
# drug_info: https://www.cancerrxgene.org/downloads/drug_data
drug_df = pd.read_csv('./data/raw_data/drug_info.csv', sep=',')
drug_df.head()

In [ ]:
# pubchem_id exclude non-numeric rows
import re
nonnumber = re.compile(r'\D+')
pubchem_id = list(set(drug_df['pubchem']))
pubchem_id = [i.split(',')[0] if "," in i else i for i in pubchem_id]
pubchem_id = [i for i in pubchem_id if re.findall(pattern=nonnumber, string=i).__len__()==0]
drug_df = drug_df[drug_df['pubchem'].isin(pubchem_id)]

In [ ]:
all_experiment.join(drug_df.set_index('drug_name'), on='DRUG_NAME', how="inner")

In [ ]:
all_experiment = all_experiment[all_experiment['CELL_LINE_NAME'].isin(celline_barcode)]
all_experiment.reset_index(inplace = True)
all_experiment.drop(columns = "index", inplace = True)
all_experiment

In [ ]:
all_experiment.to_csv("./data/processed_data/expriment.csv", sep="\t", index = None)

## Response and Pubchem ID troubleshooting

In [ ]:
import pubchempy as pcp
df = pcp.get_properties(properties=['canonical_smiles'], identifier=list(all_experiment['pubchem']),
                        namespace='cid', )
df = pd.DataFrame(df)
df[['CID']]=df[['CID']].astype(str)
df.to_csv("./data/processed_data/pubchem_id-SMILES.csv", sep='\t')
lookup_table_cid_smiles = dict(zip(df['CID'], df['CanonicalSMILES']))

In [ ]:
all_experiment = all_experiment[all_experiment['PUBCHEM_ID'].isin(pubchem_id)]
all_experiment['SMILES']=[lookup_table_cid_smiles[i] for i in all_experiment['PUBCHEM_ID']]

In [ ]:
all_experiment.shape

In [ ]:
sample_barcode = [f"{i[0]}_{i[1]}" for i in zip(all_experiment['CELL_LINE_NAME'], all_experiment['PUBCHEM_ID'])]

In [ ]:
response = pd.DataFrame()
response['sample_barcode'] = sample_barcode
response['LN_IC50'] = all_experiment['LN_IC50']
response.reset_index(drop=True, inplace=True)

**exclude outliers**

In [ ]:
from matplotlib import pyplot
ln_ic50 = all_experiment['LN_IC50'].values
df = pd.DataFrame(ln_ic50)

lower, median, upper = df.quantile([0.15,0.5,0.85]).values
IQR = upper - lower
lower_limit = lower - 1.5*IQR
upper_limit = upper + 1.5*IQR

all_experiment.loc[(all_experiment['LN_IC50'] < upper_limit.data) &
                   (all_experiment['LN_IC50'] > lower_limit.data)]

all_experiment.reset_index(drop=True, inplace=True)

In [ ]:
response.to_csv('./data/processed_data/response.csv', sep='\t', index=None)
all_experiment.to_csv('./data/processed_data/expriment.csv', sep='\t', index=None)

## Drug Feature Extraction using beta-VAE

In [ ]:
import pandas as pd
from os.path import realpath
import pandas as pd
from keras.models import load_model
import ast
import pandas as pd
import numpy as np
from rdkit import Chem, RDLogger
from rdkit.Chem import BondType

RDLogger.DisableLog("rdApp.*")

SMILE_CHARSET = '["C", "B", "F", "I", "H", "O", "N", "S", "P", "Cl", "Br"]'

bond_mapping = {"SINGLE": 0, "DOUBLE": 1, "TRIPLE": 2, "AROMATIC": 3}
bond_mapping.update(
    {0: BondType.SINGLE, 1: BondType.DOUBLE, 2: BondType.TRIPLE, 3: BondType.AROMATIC}
)
SMILE_CHARSET = ast.literal_eval(SMILE_CHARSET)

SMILE_to_index = dict((c, i) for i, c in enumerate(SMILE_CHARSET))
index_to_SMILE = dict((i, c) for i, c in enumerate(SMILE_CHARSET))
atom_mapping = dict(SMILE_to_index)
atom_mapping.update(index_to_SMILE)

BATCH_SIZE = 100
EPOCHS = 10

VAE_LR = 5e-4
NUM_ATOMS = 120  # Maximum number of atoms

ATOM_DIM = len(SMILE_CHARSET)  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 435  # Size of the latent space


def smiles_to_graph(smiles):
    # Converts SMILES to molecule object
    molecule = Chem.MolFromSmiles(smiles)

    # Initialize adjacency and feature tensor
    adjacency = np.zeros((BOND_DIM, NUM_ATOMS, NUM_ATOMS), "float32")
    features = np.zeros((NUM_ATOMS, ATOM_DIM), "float32")

    # loop over each atom in molecule
    # Ignore Pt Atom
    for atom in molecule.GetAtoms():
        if(atom.GetSymbol() == "Pt"):
            continue
        i = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        features[i] = np.eye(ATOM_DIM)[atom_type]
        # loop over one-hop neighbors
        for neighbor in atom.GetNeighbors():
            j = neighbor.GetIdx()
            bond = molecule.GetBondBetweenAtoms(i, j)
            bond_type_idx = bond_mapping[bond.GetBondType().name]
            adjacency[bond_type_idx, [i, j], [j, i]] = 1

    # Where no bond, add 1 to last channel (indicating "non-bond")
    # Notice: channels-first
    adjacency[-1, np.sum(adjacency, axis=0) == 0] = 1

    # Where no atom, add 1 to last column (indicating "non-atom")
    features[np.where(np.sum(features, axis=1) == 0)[0], -1] = 1

    return adjacency, features


def graph_to_molecule(graph):
    # Unpack graph
    adjacency, features = graph

    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    # Remove "no atoms" & atoms with no bonds
    keep_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (np.sum(adjacency[:-1], axis=(0, 1)) != 0)
    )[0]
    features = features[keep_idx]
    adjacency = adjacency[:, keep_idx, :][:, :, keep_idx]

    # Add atoms to molecule
    for atom_type_idx in np.argmax(features, axis=1):
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    # Add bonds between atoms in molecule; based on the upper triangles
    # of the [symmetric] adjacency tensor
    (bonds_ij, atoms_i, atoms_j) = np.where(np.triu(adjacency) == 1)
    for (bond_ij, atom_i, atom_j) in zip(bonds_ij, atoms_i, atoms_j):
        if atom_i == atom_j or bond_ij == BOND_DIM - 1:
            continue
        bond_type = bond_mapping[bond_ij]
        molecule.AddBond(int(atom_i), int(atom_j), bond_type)

    # Sanitize the molecule; for more information on sanitization, see
    # https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization
    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    # Let's be strict. If sanitization fails, return None
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None

    return molecule


# 注：min-Max归一化需要在分割完训练集和测试集和Validation set之后再进行

# Cached_data
cached_data = {}

# Path Define
cna_path = realpath("data/processed_data/cna.csv")
experiment_path = realpath("data/processed_data/experiment.csv")
fpkm_path = realpath("data/processed_data/fpkm.csv")
SMILES_path = realpath("data/processed_data/pubchem_id-SMILES.csv")

drug_AdjacencyTensor = []
drug_FeatureTensor = []

df = pd.read_csv(SMILES_path, sep='\t')
for i in df["CanonicalSMILES"]:
    _ad, _fe = smiles_to_graph(i)
    drug_AdjacencyTensor.append(_ad)
    drug_FeatureTensor.append(_fe)

drug_AdjacencyTensor = np.array(drug_AdjacencyTensor)
drug_FeatureTensor = np.array(drug_FeatureTensor)

vae = load_model("utils/drug-molecule-generation-with-VAE",
               compile=False)

z_mean, _ = vae.encoder.predict([drug_AdjacencyTensor[:1000], drug_FeatureTensor[:1000]])

In [ ]:
df[df['CID'].duplicated()]

In [ ]:
drug_feature_df = pd.DataFrame(data = z_mean)

In [ ]:
drug_feature_df.head(5) # 435-dim vector

## CNA 和 Gene-Expression dimension reduction

**CNA and FPKM data integration using SNF**

SNFtool: https://doi.org/10.1038/nmeth.2810

R code is in current folder/SNL_integration.R

In [ ]:
!R CMD BATCH ./SNL_integration.R

In [ ]:
similarity_df = pd.read_csv("data/processed_data/simlilarity_matrix.csv")

In [ ]:
similarity_df.drop(columns=['Unnamed: 0'], inplace=True)
similarity_df.head()

In [ ]:
celline_feature = {}
for i, celline in enumerate(similarity_df.columns):
    celline_feature[celline] = np.array(similarity_df.iloc[i].values)

In [ ]:
s = []
for i in response['sample_barcode']:
    celline_name, pubchem_id = i.split('_')
    celline_feature_array = celline_feature[celline_name]
    drug_feature_array = drug_feature_df.loc[int(pubchem_id)]
    combined_feature = np.hstack([celline_feature_array, drug_feature_array])
    s.append(combined_feature)
    

In [ ]:
np.array(s).shape # 866+435=1301-dim vector

In [ ]:
np.save("./data/processed_data/all_feature", np.array(s))

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = np.load('./data/processed_data/all_feature.npy')
response = pd.read_csv('./data/processed_data/response.csv', sep='\t')


In [ ]:
import tensorflow as tf
from tensorflow.data import Dataset
from sklearn.preprocessing import MinMaxScaler

In [ ]:
features = tf.constant(data)
labels = response['LN_IC50'].values


In [ ]:
scaler = MinMaxScaler()
labels = labels.reshape(-1,1)
scaler.fit(labels)
labels = scaler.transform(labels)

Other Method


In [ ]:
from sklearn.model_selection import train_test_split
data = np.load('./data/processed_data/all_feature.npy')
response = pd.read_csv('./data/processed_data/response.csv', sep='\t')
#features = tf.constant(data)
labels = response['LN_IC50'].values
scaler = MinMaxScaler()
labels = labels.reshape(-1,1)
scaler.fit(labels)
labels = scaler.transform(labels)
#labels = np.squeeze(1)
#labels = tf.constant(labels)
# dataset = Dataset.from_tensors((features, labels))


X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.losses import MeanSquaredError, BinaryCrossentropy
from keras import Model
from keras.metrics import Accuracy, AUC
import pandas as pd
from keras.models import Sequential
from keras import layers
import numpy as np
from tensorflow.data import Dataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

data = np.load('./data/processed_data/all_feature.npy')
response = pd.read_csv('./data/processed_data/response.csv', sep='\t')
#features = tf.constant(data)
labels = response['LN_IC50'].values
scaler = MinMaxScaler()
labels = labels.reshape(-1,1)
scaler.fit(labels)
labels = scaler.transform(labels)
# labels = np.squeeze(1)
#labels = tf.constant(labels)
# dataset = Dataset.from_tensors((features, labels))

y = []
for i in labels:
    if (i[0]<=0.67):
        y.append(1)
    else:
        y.append(0)
labels = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, 
                                                    random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
y_pred = clf.predict(X_test)
precision_score(y_test, y_pred)


In [ ]:
recall_score(y_test, y_pred)


## MOFA analysis

In [ ]:
from model.data import Dataset
import pandas as pd
import numpy as np
import muon

In [ ]:
_data = Dataset()
procssed_data = _data.return_data()

In [ ]:
gene_expression = procssed_data['omics_data']['gene_expression']
cnv = procssed_data['omics_data']['cnv']
response = procssed_data['response']
experiment = procssed_data['experiment']

In [ ]:
cnv

In [ ]:
gene_expression

In [ ]:
selected_experiment = experiment[['SAMPLE_BARCODE', 'CELL_LINE_NAME', 'DRUG_NAME', 'pubchem', 'SMILES', 'AUC', 'Z_SCORE', 'LN_IC50']]

In [ ]:
selected_experiment

Prepare dataset with columns # The input needs to be a data.frame with columns ["sample","feature","view","group","value"]


In [ ]:
gene_expression_stakced = gene_expression.stack()
cnv_stacked = cnv.stack()
prepared_df = pd.DataFrame([[cll, symbol, 'gene_expression', '*', gene_expression_stakced[i]] for i, (cll, symbol) in enumerate(list(gene_expression_stakced.index))]+
                           [[cll, symbol, 'cnv', '*', cnv_stacked[i]] for i, (cll, symbol) in enumerate(list(cnv_stacked.index))],
                           columns=["sample","feature","view","group","value"])

In [ ]:
prepared_df.drop_duplicates(["sample","feature","view","group"], keep = "last", inplace=True)

In [ ]:
from mofapy2.run.entry_point import entry_point
###########################
## Initialise MOFA model ##
###########################


## (1) initialise the entry point
ent = entry_point()
## (2) Set data options
# - scale_views: if views have very different ranges, one can to scale each view to unit variance
ent.set_data_options(
	scale_views = False
)

# (3) Set data using the data frame format
ent.set_data_df(prepared_df)

# using default values
ent.set_model_options()

# using personalised values
ent.set_model_options(
	factors = 10, 
	spikeslab_weights = True, 
	ard_weights = True
)

In [ ]:
## (5) Set training options ##
# - iter: number of iterations
# - convergence_mode: "fast", "medium", "slow". Fast mode is usually good enough.
# - dropR2: minimum variance explained criteria to drop factors while training. Default is None, inactive factors are not dropped during training
# - gpu_mode: use GPU mode? this functionality needs cupy installed and a functional GPU, see https://biofam.github.io/MOFA2/gpu_training.html
# - seed: random seed

# using default values
ent.set_train_options()

# using personalised values
ent.set_train_options(
	iter = 100, 
	convergence_mode = "fast", 
	dropR2 = None, 
	gpu_mode = False, 
	seed = 42
)

####################################
## Build and train the MOFA model ##
####################################

# Build the model 
ent.build()

# Run the model
ent.run()

####################
## Save the model ##
####################

outfile = "./test.hdf5"

# - save_data: logical indicating whether to save the training data in the hdf5 file.
# this is useful for some downstream analysis in R, but it can take a lot of disk space.
ent.save(outfile, save_data=True)

In [ ]:

#########################
## Downstream analysis ##
#########################

# Check the mofax package for the downstream analysis in Python: https://github.com/bioFAM/mofax
# Check the MOFA2 R package for the downstream analysis in R: https://www.bioconductor.org/packages/release/bioc/html/MOFA2.html
# All tutorials: https://biofam.github.io/MOFA2/tutorials.html

# Extract factor values (a list with one matrix per sample group)
factors = ent.model.nodes["Z"].getExpectation()

# Extract weights  (a list with one matrix per view)
weights = ent.model.nodes["W"].getExpectation()

# Extract variance explained values
r2 = ent.model.calculate_variance_explained()

# Interact directly with the hdf5 file
import h5py
f = h5py.File(outfile, 'r')
f.keys()


In [ ]:
import seaborn as sns
sns.heatmap(data=weights[1])

In [ ]:

# Extract factors
# f["expectations"]["Z"]["*"].value

# Extract weights
#f["expectations"]["W"]["gene_expression"].value
#f["expectations"]["W"]["cnv"].value

# Extract variance explained estimates
print(f["variance_explained"]["r2_per_factor"])
print(f["variance_explained"]["r2_total"])

In [ ]:
pd.read_csv("data/processed_data/simlilarity_matrix.csv")

In [ ]:
plt.hist(x = selected_experiment['LN_IC50'], bins=100)

# CRTP Dataset Overview

In [ ]:
experiment = pd.read_csv("data/raw_data/CTRPv2.0/v20.data.curves_post_qc.txt", sep='\t')
meta_celline = pd.read_csv("data/raw_data/CTRPv2.0/v20.meta.per_cell_line.txt", sep="\t")
meta_experiment = pd.read_csv("data/raw_data/CTRPv2.0/v20.meta.per_experiment.txt", sep='\t')
meta_compound = pd.read_csv("data/raw_data/CTRPv2.0/v20.meta.per_compound.txt", sep='\t')

In [ ]:
meta_celline = meta_celline[['master_ccl_id', 'ccl_name']]
meta_compound = meta_compound[['master_cpd_id', 'cpd_name', 'cpd_smiles']]
meta_experiment = meta_experiment[['experiment_id','master_ccl_id']]
experiment = experiment[['experiment_id', 'area_under_curve', 'master_cpd_id']]

In [ ]:
experiment = experiment.join(meta_experiment.set_index('experiment_id'), on='experiment_id',how='inner')
experiment = experiment.join(meta_compound.set_index('master_cpd_id'), on='master_cpd_id',how='inner')
experiment = experiment.join(meta_celline.set_index('master_ccl_id'), on='master_ccl_id', how='inner')

In [ ]:
experiment = experiment[['experiment_id', 'master_cpd_id', 'cpd_name', 'cpd_smiles',
                         'master_ccl_id', 'ccl_name', 'area_under_curve']] # reorder

In [ ]:
experiment.tail()

In [1]:
from config_path import *
from model.data import Dataset

ds = Dataset(feature_contained=['cnv', 'mutation', 'gene_expression', 'methylation'])

Loading Copy Number Abberation Data...
Loading Copy Number Abberation Data Done
Loading Gene Expression Data...
Loading Gene Expression Data Done
Loading Methylation Data...


/Users/sunzehui/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Loading Methylation Data Done
Loading Mutations Data...
Loading Mutations Data Done
Begin loading drug data...
After filtering, GDSC has tested 398 drugs 
 CTRP has tested 470 drugs
After combining, unique cid number is 749


/Users/sunzehui/miniconda3/envs/tf/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:6300: RuntimeWarning: overflow encountered in _ncf_cdf
  return _boost._ncf_cdf(x, dfn, dfd, nc)


Drug data loaded
Loading GDSC Experiment Data...
Loading Experiment Data Done
Index(['DATASET', 'CELL_LINE_NAME', 'DRUG_NAME', 'LN_IC50', 'AUC'], dtype='object')
Begin Preprocessing Experiment!
Select Overlapping Cellines...
Select Overlapping Cellines with available PubChem CIDs...
Create Unique Sample Barcode...
Exclude response value...
Experiment Done!


In [35]:
import pandas as pd
import numpy as np
from config_path import *
def load_data_methylation(data_path) -> pd.DataFrame:
    print("Loading Methylation Data...")
    met_df = pd.read_csv(data_path, sep = '\t', index_col=0)
    temp = pd.read_excel(RAW_SENTRIX2SAMPLE_GDSC_PATH, sheet_name=0)
    temp['Sentrix_Barcode'] = list("_".join([i,j]) for i, j in zip(temp['Sentrix_ID'].astype(str), temp['Sentrix_Position']))
    tb = dict(zip(temp['Sentrix_Barcode'], temp['Sample_Name']))
    met_df.rename(columns=tb, inplace=True)
    met_df = met_df.T
    met_df = met_df.reset_index().drop_duplicates(subset='index', keep='first').rename(columns={'index': 'celline'})
    met_df = met_df.set_index('celline')
    print("Loading Methylation Data Done")
    return met_df

In [36]:
df=load_data_methylation(RAW_METHYLATION_GDSC_PATH)

Loading Methylation Data...


/Users/sunzehui/miniconda3/envs/tf/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Loading Methylation Data Done


In [38]:
df

,chr1:10003165-10003585,chr1:100315420-100316009,chr1:100435297-100436070,chr1:100503482-100504404,chr1:10057121-10058108,chr1:100598130-100598754,chr1:100731632-100731999,chr1:100817709-100818899,chr1:10092532-10093404,chr1:101004471-101005885,...,chr9:99179479-99181723,chr9:99212164-99213507,chr9:99328810-99329764,chr9:99380814-99382236,chr9:99416904-99417592,chr9:99540025-99540472,chr9:99616401-99616940,chr9:99775098-99776102,chr9:99800856-99801724,chr9:99962136-99962426
celline,,,,,,,,,,,,,,,,,,,,,
MC-CAR,0.192212,0.203820,0.277914,0.190930,0.544059,0.273826,0.246309,0.132243,0.265601,0.552313,...,0.161748,0.140332,0.171117,0.504871,0.593335,0.295142,0.396794,0.264053,0.301407,0.208931
PFSK-1,0.187603,0.207652,0.400146,0.195871,0.764898,0.265925,0.245317,0.081613,0.246907,0.782697,...,0.156671,0.179353,0.665262,0.202245,0.553815,0.223422,0.401027,0.311693,0.290512,0.155249
A673,0.210185,0.222116,0.264730,0.243298,0.415485,0.308657,0.258123,0.163548,0.281540,0.459978,...,0.161361,0.197860,0.216423,0.431386,0.546775,0.242035,0.446645,0.333607,0.319258,0.307825
ES3,0.205614,0.227570,0.303641,0.250454,0.599275,0.284652,0.255257,0.183600,0.249813,0.346111,...,0.139215,0.200593,0.204809,0.360808,0.537972,0.255180,0.513135,0.352037,0.317108,0.267645
ES5,0.204037,0.204111,0.243705,0.227564,0.389168,0.279121,0.235306,0.144075,0.249098,0.300132,...,0.147075,0.205460,0.449984,0.340790,0.469317,0.257921,0.528488,0.337319,0.280758,0.192772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MO,0.202672,0.240531,0.293938,0.195461,0.623589,0.285769,0.249944,0.143821,0.246995,0.490227,...,0.129995,0.125894,0.099382,0.348599,0.524072,0.634188,0.432816,0.284111,0.429171,0.467794
SNU-5,0.196493,0.187575,0.148768,0.187911,0.427652,0.248952,0.264237,0.126252,0.181884,0.705789,...,0.102420,0.190474,0.119517,0.224722,0.572469,0.257184,0.315729,0.302024,0.284147,0.308154
MINO,0.201091,0.171479,0.236005,0.203405,0.462792,0.280228,0.246454,0.130319,0.230951,0.534987,...,0.158141,0.092234,0.104812,0.260832,0.518584,0.292160,0.287092,0.245871,0.643805,0.314371


In [22]:
df = df.reset_index().drop_duplicates(subset=['index'])

In [39]:
_predicted = np.load("./_predict.npy")
_sample = np.load("./_sample.npy")

In [48]:
len(_sample)

17306

In [50]:
for i, j in zip(_sample, _predicted):
    print(i, " ", j[0])

OE21_42623951   0.0012725707
NCI-H1963_53469448   0.936808
647-V_148124   0.00010219075
NCI-H1155_56645356   0.15767367
CP66-MEL_59472121   0.7584004
SK-HEP-1_44632017   0.045382377
EW-11_72201027   4.327651e-05
SUP-B15_25262965   0.00051338435
HOS_3062316   0.0061830757
PANC-03-27_208908   0.99686193
EW-12_25099184   0.048246063
CAS-1_2130404   0.0014042953
22RV1_5330286   0.05219027
GA-10_25227436   0.000115103154
MC116_24788740   0.08605056
CW-2_126565   0.06923866
KMH-2_16095342   0.00059868535
HT-1080_992586   0.055667944
G-361_4212   0.00019733513
TT2609-C02_57345410   0.0014032299
T-T_51039095   0.8834034
SAT_24771867   0.0042907842
DBTRG-05MG_11707110   0.73778325
ESS-1_57519544   0.07537825
NCI-H2122_126941   0.12025746
LB996-RCC_91668541   0.0057346737
SNU-81_24958200   0.00083206006
MDA-MB-330_46931953   0.00059255696
GI-1_135564749   0.00023587367
RKN_51039095   0.21614239
D-423MG_24964624   0.00039518735
HT-1197_56649450   0.1514613
RPMI-8402_16681698   0.05219027
CAL-54_2